In [1]:
import cv2
import numpy as np
import PIL as Image
import os
import time


def deteccaoSSD(network,imagem_path,tamanho=300,conf_min = 0.4):
    
    imagem = cv2.imread(imagem_path)
    
    (h,w) = imagem.shape[:2]

    blob = cv2.dnn.blobFromImage(cv2.resize(imagem,(tamanho,tamanho)),1.0,(tamanho,tamanho),(104.0,117.0,123.0))
    
    network.setInput(blob)
    
    deteccoes = network.forward()

    for i in range(0,deteccoes.shape[2]):
    
        confianca = deteccoes[0,0,i,2]
    
        if confianca > conf_min:
            
            text_conf = "{:2f}%".format(confianca*100)
    
            box = deteccoes[0,0,i,3:7]*np.array([w,h,w,h])    
            
            (startX,startY,endX,endY) = box.astype(int)

            #Copia a área de interesse na imagem
            roi = imagem[startY:endY,startX:endX]
    
            cv2.rectangle(imagem,(startX,startY),(endX,endY),(0,255,0),2)

            #Escreve qual a confiança de ser realmente um rosto
            #cv2.putText(imagem,text_conf,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
      
    rosto = cv2.resize(roi,(60,80))
    rosto = cv2.cvtColor(rosto,cv2.COLOR_BGR2GRAY)
    
    return rosto, imagem, confianca
    

def get_image_data():

    arquivo_modelo = 'Materiais\\weights\\weights\\res10_300x300_ssd_iter_140000.caffemodel'
    arquivo_prototxt = 'Materiais\\weights\\weights\\deploy.prototxt.txt'
    network = cv2.dnn.readNetFromCaffe(arquivo_prototxt,arquivo_modelo)

    
    paths = [os.path.join('Materiais\\datasets\\datasets\\Face dataset\\train',f) for f in os.listdir('Materiais\\datasets\\datasets\\Face dataset\\train')]

    faces = []
    ids = []

    for path in paths:
        
        rosto,imagem_original,confianca = deteccaoSSD(network,path,300,0.4) 
        
        id = int(os.path.split(path)[1].split("-")[0])
        
        ids.append(id)
        faces.append(rosto)

    return np.array(ids), faces


def teste_reconhecimento(imagem_teste):

    '''CARREGA MODELO PARA DETECTAR FACES'''
    arquivo_modelo = 'Materiais\\weights\\weights\\res10_300x300_ssd_iter_140000.caffemodel'
    arquivo_prototxt = 'Materiais\\weights\\weights\\deploy.prototxt.txt'
    network = cv2.dnn.readNetFromCaffe(arquivo_prototxt,arquivo_modelo)

    '''DETECTA FACES'''
    rosto,imagem_original, confianca = deteccaoSSD(network,imagem_teste,300,0.4)

    '''CARREGA MODELO PARA RECONHECER FACES'''
    lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
    lbph_classifier.read('lbph_classifier.yml')

    '''RECONHECE FACES'''
    previsao = lbph_classifier.predict(rosto)
    saida_esperada = int(os.path.split(imagem_teste)[1].split("-")[0])

    prev = ""

    '''CASO A CONFIANÇA ULTRAPASSE 1000 SIGNIFICA QUE A IMAGEM ESTÁ COM UMA CONFIANÇA BAIXA, LOGO A PREVISÃO SE TORNA INDEFINIDA'''
    if confianca > 1000:
        prev = "INDEFINIDO"
    else:
        prev = str(previsao[0])

    
    #cv2.putText(imagem_original,'Precisao: '+str(previsao[1]),(10,70),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
    cv2.putText(imagem_original,'Prev: '+ prev,(10,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,255,0))
    cv2.putText(imagem_original,'Exp: '+ str(saida_esperada),(10,50),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,255,0))

    return imagem_original,previsao,saida_esperada,previsao[1]
    
    

In [6]:
'''
TREINA MODELO
'''

ids, faces = get_image_data()

lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
lbph_classifier.train(faces,ids)

lbph_classifier.write('lbph_classifier.yml')

In [12]:
start_time = time.time()

#imagem_teste = 'Materiais\\datasets\\datasets\\Face dataset\\test\\11-10.jpg'
imagem_teste = 'Materiais\\datasets\\datasets\\Face dataset\\test\\2-02.jpg'
imagem_original,previsao,saida_esperada,confianca = teste_reconhecimento(imagem_teste)


cv2.imshow("Rosto",imagem_original)

cv2.waitKey(0) 
cv2.destroyAllWindows()  


end_time = time.time()

elapsed_time = end_time - start_time